In [45]:
#installing kaggle library
!pip install kaggle

**Upload kaggle json file**

In [89]:
#configuring the path of kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

**Importing twitter sentiment dataset**

In [90]:
# API to fetch dataset from kaggle
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [91]:
#extracting the compressed the dataset

from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


**Importing the dependencies**

In [92]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [93]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [94]:
#printing the stopwords in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

**Data Collection & Processing**

In [95]:
# loading the data from csv file to pandas dataframe
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')


In [96]:
#check in the number of rows and columns
twitter_data.shape

(1599999, 6)

In [97]:
#printing the first 5 rows of the dataset
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [98]:
#naming the columns and reading the dataset again
column_names = ['target','id','date','flag','user','text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names = column_names, encoding='ISO-8859-1')


In [99]:
#check in the number of rows and columns
twitter_data.shape

(1600000, 6)

In [100]:
#printing the first 5 rows of the dataset
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [101]:
#counting the number of missing values in the dataset
twitter_data.isnull().sum()

,0
target,0
id,0
date,0
flag,0
user,0
text,0


In [102]:
#checking the distribution of target column
twitter_data['target'].value_counts()

,count
target,
0,800000
4,800000


**convert the target "4" to "1"**

In [103]:
twitter_data.replace({"target":{4:1}}, inplace =True)

In [104]:
#checking the distribution of target column
twitter_data['target'].value_counts()

,count
target,
0,800000
1,800000


0--> **Negative tweets** <br>
1--> **Positive tweets**

**Stemming**

Stemming is the process of reducing a word to its Root word

Example: actor,actress,acting = act

In [105]:
port_stem = PorterStemmer()

In [106]:
def stemming(content):

  stemmed_content = re.sub('[^a-zA-Z]','',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ''.join(stemmed_content)

  return stemmed_content

In [107]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [108]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoothttptwitpiccomyzlawwwthatsabummeryou...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,isupsetthathecantupdatehisfacebookbytextingita...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichanidivedmanytimesfortheballmanagedtosave...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,mywholebodyfeelsitchyandlikeitsonfir
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclassnoitsnotbehavingatallimmadwhyam...


In [109]:
print(twitter_data['stemmed_content'])

0          switchfoothttptwitpiccomyzlawwwthatsabummeryou...
1          isupsetthathecantupdatehisfacebookbytextingita...
2          kenichanidivedmanytimesfortheballmanagedtosave...
3                       mywholebodyfeelsitchyandlikeitsonfir
4          nationwideclassnoitsnotbehavingatallimmadwhyam...
                                 ...                        
1599995           justwokeuphavingnoschoolisthebestfeelingev
1599996    thewdbcomverycooltohearoldwaltinterviewshttpbl...
1599997         areyoureadyforyourmojomakeoveraskmefordetail
1599998     happythbirthdaytomybooofallltimetupacamarushakur
1599999    happycharitytuesdaythenspccsparkscharityspeaki...
Name: stemmed_content, Length: 1600000, dtype: object


In [110]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [111]:
#separating the data and label
x = twitter_data['stemmed_content'].values
y = twitter_data['target'].values

In [112]:
print(x)

['switchfoothttptwitpiccomyzlawwwthatsabummeryoushouldagotdavidcarrofthirddaytodoitd'
 'isupsetthathecantupdatehisfacebookbytextingitandmightcryasaresultschooltodayalsoblah'
 'kenichanidivedmanytimesfortheballmanagedtosavetherestgooutofbound' ...
 'areyoureadyforyourmojomakeoveraskmefordetail'
 'happythbirthdaytomybooofallltimetupacamarushakur'
 'happycharitytuesdaythenspccsparkscharityspeakinguphh']


In [113]:
print(y)

[0 0 0 ... 1 1 1]


**splitting the data to training data and testing data**

In [114]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,stratify = y,random_state=2)

In [115]:
print(x.shape,x_train.shape,x_test.shape)

(1600000,) (1280000,) (320000,)


In [116]:
print(x_train)

['abouttowatchsawivanddrinkalilwin' 'hatermagazineimin'
 'eventhoughitsmyfavouritedrinkithinkitsthevodkaandcokethatwipesmymindallthetimethinkimgonnahavetofindanewdrink'
 ... 'iseagerformondayafternoon'
 'hopeeveryoneandtheirmotherhadagreatdaycantwaittohearwhattheguyshaveinstoretomorrow'
 'ilovewakinguptofolgerstoobadmyvoicewasdeeperthanhi']


In [117]:
print(x_test)

['mmangenmdoingfineihaventhadmuchtimetochatontwitterhubbyisbackforthesummeramptendstodominatemyfreetim'
 'atahsmayshowwruthkimampgeoffreysanhueza'
 'ishataramaybeitwasonlyabayareathangdammit' ...
 'destinineverthelesshoorayformembersandhaveawonderfulandsafetrip'
 'notfeelingtoowel' 'supersandrothanky']


In [118]:
#converting the textual data into numerical data
from sklearn.feature_extraction.text import TfidfVectorizer

# Instantiate the TfidfVectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the training data
x_train = vectorizer.fit_transform(x_train)

# Transform the test data using the fitted vectorizer
x_test = vectorizer.transform(x_test)


In [119]:
print(x_train)

  (0, 4269)	1.0
  (1, 370478)	1.0
  (2, 259878)	1.0
  (3, 544662)	1.0
  (4, 1125692)	1.0
  (5, 183084)	1.0
  (6, 655943)	1.0
  (7, 427969)	1.0
  (8, 372293)	1.0
  (9, 414371)	1.0
  (10, 453206)	1.0
  (11, 946311)	1.0
  (12, 1165280)	1.0
  (13, 334893)	1.0
  (14, 1029544)	1.0
  (15, 1251389)	1.0
  (16, 26694)	1.0
  (17, 620642)	1.0
  (18, 371055)	1.0
  (19, 166384)	1.0
  (20, 939661)	1.0
  (21, 392755)	1.0
  (22, 288391)	1.0
  (23, 202689)	1.0
  (24, 1253472)	1.0
  :	:
  (1279975, 97244)	1.0
  (1279976, 1110776)	1.0
  (1279977, 938969)	1.0
  (1279978, 783480)	1.0
  (1279979, 800934)	1.0
  (1279980, 865418)	1.0
  (1279981, 587370)	1.0
  (1279982, 952975)	1.0
  (1279983, 412833)	1.0
  (1279984, 81684)	1.0
  (1279985, 59066)	1.0
  (1279986, 1015971)	1.0
  (1279987, 777243)	1.0
  (1279988, 1218709)	1.0
  (1279989, 450972)	1.0
  (1279990, 465151)	1.0
  (1279991, 249222)	1.0
  (1279992, 1169847)	1.0
  (1279993, 1170137)	1.0
  (1279994, 75555)	1.0
  (1279995, 1106989)	1.0
  (1279996, 561111)	1

In [120]:
print(x_test)

  (22, 137709)	1.0
  (36, 790487)	1.0
  (55, 1232068)	1.0
  (104, 806213)	1.0
  (316, 675309)	1.0
  (317, 183110)	1.0
  (326, 530902)	1.0
  (343, 483231)	1.0
  (356, 1004786)	1.0
  (405, 294045)	1.0
  (411, 754846)	1.0
  (412, 671785)	1.0
  (503, 922733)	1.0
  (575, 484365)	1.0
  (576, 68315)	1.0
  (585, 553737)	1.0
  (597, 437993)	1.0
  (704, 437477)	1.0
  (730, 522522)	1.0
  (735, 1190472)	1.0
  (763, 201539)	1.0
  (851, 230654)	1.0
  (887, 945330)	1.0
  (905, 1080766)	1.0
  (965, 64114)	1.0
  :	:
  (319388, 469685)	1.0
  (319391, 363430)	1.0
  (319401, 69390)	1.0
  (319403, 1010168)	1.0
  (319419, 435823)	1.0
  (319442, 1154002)	1.0
  (319462, 331998)	1.0
  (319474, 363236)	1.0
  (319550, 1020181)	1.0
  (319573, 1087679)	1.0
  (319595, 377142)	1.0
  (319630, 416243)	1.0
  (319632, 324586)	1.0
  (319640, 333810)	1.0
  (319649, 564791)	1.0
  (319673, 473916)	1.0
  (319679, 363450)	1.0
  (319687, 874322)	1.0
  (319700, 937693)	1.0
  (319712, 230590)	1.0
  (319779, 553484)	1.0
  (319838

**Training the machine learning model**

**Logistic regression**

In [121]:
model = LogisticRegression(max_iter =1000)
model.fit(x_train,y_train)

LogisticRegression(max_iter=1000)

**Model Evaluation**

**Accuracy Score**

In [122]:
#accuracy score on the training data
x_train_prediction = model.predict(x_train)
training_data_accuracy = accuracy_score(y_train,x_train_prediction)

In [123]:
print("accuracy score on the training data:",training_data_accuracy)


accuracy score on the training data: 0.99812265625


In [124]:
#accuracy score on the test data
x_test_prediction = model.predict(x_test)
test_data_accuracy = accuracy_score(y_test,x_test_prediction)

In [125]:
print("accuracy score on the test data:",test_data_accuracy)

accuracy score on the test data: 0.512653125


In [126]:
import pickle

In [127]:
filename = 'trained_model.sav'
pickle.dump(model,open(filename,'wb'))

**Using the saved model for future predictions**

In [128]:
#loading the saved model
loaded_model = pickle.load(open('/content/trained_model.sav','rb'))


In [129]:
x_new = x_test [200]
print(y_test[200])

prediction = model.predict(x_new)
print(prediction)

if (prediction[0] == 0):
  print("Negative Tweet") # indented to signify code block associated with if statement

else:
  print("Positive Tweet")

1
[1]
Positive Tweet


In [135]:
!pip install flask-ngrok
!pip install Flask


In [139]:

from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
import pickle

model = pickle.load(open('/content/trained_model.sav', 'rb'))

import pickle

# Assuming your trained model is called 'model'
# Save the model to a file
filename = '/content/trained_model.sav'
with open(filename, 'wb') as file:
    pickle.dump(model, file)

# To load it later
with open('/content/trained_model.sav', 'rb') as file:
    model = pickle.load(file)



In [142]:
!pip install dash
!pip install dash-bootstrap-components


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 kB 1.6 MB/s eta 0:00:00


In [145]:
from dash import dcc, html


In [146]:
!pip install flask dash pyngrok


In [ ]:
from flask import Flask
from pyngrok import ngrok
from dash import dcc, html
import dash
import plotly.graph_objs as go
import pandas as pd
from dash.dependencies import Input, Output

# Create Flask server
app = Flask(__name__)

# Optional: Add a route for the root
@app.route('/')
def index():
    return "Welcome to the Sentiment Dashboard. Go to /dashboard/ to see the live data."

# Create Dash app
dash_app = dash.Dash(__name__, server=app, routes_pathname_prefix='/dashboard/')

dash_app.layout = html.Div([
    dcc.Graph(id='live-update-graph'),
    dcc.Interval(id='interval-component', interval=1*1000, n_intervals=0)
])

# Real-time sentiment trends data (Dummy data)
@dash_app.callback(
    Output('live-update-graph', 'figure'),
    [Input('interval-component', 'n_intervals')]
)
def update_graph_live(n):
    # Generate dummy sentiment data
    df = pd.DataFrame({
        'Time': pd.date_range(start='1/1/2024', periods=50, freq='S'),
        'Sentiment': [1, 0] * 25
    })
    fig = go.Figure([go.Scatter(x=df['Time'], y=df['Sentiment'], mode='lines')])
    return fig

# Expose the app to the public via ngrok
public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")

# Start Flask app
if __name__ == '__main__':
    app.run()


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Public URL: NgrokTunnel: "https://c95c-34-67-231-24.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [24/Sep/2024 05:48:06] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [24/Sep/2024 05:48:07] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [24/Sep/2024 05:52:15] "GET /dashboard/ HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Sep/2024 05:52:16] "GET /dashboard/_dash-component-suites/dash/deps/polyfill@7.v2_18_1m1727154794.12.1.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Sep/2024 05:52:16] "GET /dashboard/_dash-component-suites/dash/deps/react@16.v2_18_1m1727154794.14.0.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Sep/2024 05:52:16] "GET /dashboard/_dash-component-suites/dash/deps/prop-types@15.v2_18_1m1727154794.8.1.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Sep/2024 05:52:16] "GET /dashboard/_dash-compon